In [8]:
def three_end_mapping(samgen,GFFgen_filename):
    from BCBio import GFF
    from Bio import Seq
    outputdata1={}
    outputdata2={}
    GFFgen=GFF.parse(GFFgen_filename)
    for sequence in GFFgen:
        outputdata1[sequence.id]=[0 for x in range(len(sequence))]
        outputdata2[sequence.id]=[0 for x in range(len(sequence))]
        for read in samgen:
            if read[0][0] == '@':   # Ignore header lines.
                continue
            if read[1] == '4':      # A bowtie mismatch. These are the same as those previously saved in the files bowtie output as reads without matches and reads exceeding the -m limit. No need to write as fastq.
                continue            
            chrom = read[2]             # chromosome identified for read in bowtie
            readid = read[0]            # read id
            startp = int(read[3]) - 1    # start position. Need to subtract 1 since genomic sequence starts at 1, but list structures for read summing start at 0.
            seq = Seq.Seq(read[9])      # sequence of the read
            length = len(seq)           # length of read
            if (read[1] == '16'):
                start = startp
                outputdata2[chrom][start] += 1
            if (read[1] == '0'):
                start = startp + length - 1
                outputdata1[chrom][start] += 1
    return [outputdata1,outputdata2]  

def readcounts_to_rpm(readcounts):
    for chrom in readcounts.keys():
        totalread = sum(readcounts[chrom])
        for position in range(len(readcounts[chrom])):
            readcounts[chrom][position]/=float(totalread)
            readcounts[chrom][position]*=1E6

# makes density file 
# the last letter "f" in writecountsf means float
def writecountsf(rpm,filepath):  #Resultlist it is the list of counts, filestring is the file prefix for each chr to be written.
    import struct
    f2=open(filepath+"keys","w")
    for chrom in rpm.keys():
        f=open(filepath+"genome","wb")
        # used to be (filepath+chrom,"wb") but server fail to show a file with a name containing "|"
        # rename "genome" to real genome name manually 
        for position in rpm[chrom]:
            f.write(struct.pack("f",position))
            # struct.pack to make binary based on the format indicated 
            # here, the indication of format is "f", which means float
        f.close()
        f2.write(chrom+"\n")
    f2.close()

# makes wig file
def countstowig(rpm,filestring):
    import random
    f=open(filestring+".wig","w")
    filestring=filestring.partition("_")[0][-3:]

    random.seed(filestring)
    c1=random.randint(0,255)
    random.seed(filestring+"xxx")
    c2=random.randint(0,255)
    random.seed(filestring+"000")
    c3=random.randint(0,255)

    f.write("track name=tracklabel viewLimits=-5:5 color="+str(c1)+','+str(c2)+','+str(c3)+"\n")
    for chrom in rpm.keys():
        if chrom[0:3]=='chr':
            f.write("fixedStep  chrom="+chrom+"  start=1  step=1\n")
        else:
            f.write("fixedStep  chrom=\""+chrom+"\"  start=1  step=1\n")	# Note the extra '' around chrom added for E coli. Should be fine for yeast. Actually it's not. So new code here to detect that.


        for position in rpm[chrom]:
            f.write(str(position)+"\n")
    f.close()

In [3]:
fastq_path = '/home/kazuki/^profilingdata/20160420/'
filepath = '/home/kazuki/^profilingdata/20160420_reanalyze/'
files = ['ks'+str(x) for x in range(5,23)]
bowtie_input_extension = '-trimmed.fastq'
mapping = '_3map'
gene_type = '_gene'

In [13]:
######################################################
from datetime import datetime
import os

skewer = '/home/kazuki/^data_analysis/analtools/skewer022/skewer-0.2.2-linux-x86_64'
option = '-Q 10'
adapter = 'CTGTAGGCACCATCAAT'

log_file = filepath+'log_skewer.txt'
###
logs = open(log_file, 'w')
present = str(datetime.now())
logs.write(present+'\n')
logs.close()
###


for f in files:
    print 'skewer for ' + f
    file_path = filepath+f+'.fastq'
    
    command = '%s %s -x %s %s >> %s' % (
        skewer,
        option,
        adapter,
        file_path,
        log_file
        ) 
    
    os.system(command)
    
print 
print 
for line in open(log_file, 'r'):
    print line[:-1]

skewer for ks5
skewer for ks6
skewer for ks7
skewer for ks8
skewer for ks9
skewer for ks10
skewer for ks11
skewer for ks12
skewer for ks13
skewer for ks14
skewer for ks15
skewer for ks16
skewer for ks17
skewer for ks18
skewer for ks19
skewer for ks20
skewer for ks21
skewer for ks22


2019-06-03 09:25:25.448889
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	CTGTAGGCACCATCAAT
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- mean quality threshold (-Q):		10
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Sanger/Illumina 1.8+ FASTQ (auto detected)
-- minimum overlap length for adapter detection (-k):	3
Mon Jun  3 09:25:25 2019 >> started

Mon Jun  3 09:26:04 2019 >> done (39.127s)
18495146 reads processed; of these:
    6792 ( 0.04%) reads filtered ou

In [14]:
######################################################
######################################################
from datetime import datetime
import os

inpath = filepath
outpath = filepath

log_file = inpath+'log_bowtie_ks48.txt'
###
logs = open(log_file, 'w')
present = str(datetime.now())
logs.write(present+'\n')
logs.close()
###

path_l = outpath + 'alignments/ladder/'
path_t = outpath + 'alignments/tRNA/'
path_r = outpath + 'alignments/rRNA/'
path_chr = outpath + 'alignments/chr/'
path_temp = outpath + 'tmpds/'

if not os.path.exists(path_l):
    os.makedirs(path_l)
if not os.path.exists(path_t):
    os.makedirs(path_t)
if not os.path.exists(path_r):
    os.makedirs(path_r)
if not os.path.exists(path_chr):
    os.makedirs(path_chr)
if not os.path.exists(path_temp):
    os.makedirs(path_temp)
    #print 'really? there is no tmpds folder?'

    ###uses bowtie installed in server
for fname in files:
    print fname

    bowtie_command = 'bowtie -v 2 -y -m 1 -a --best --strata -S -p 2 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
        # The `--strata` and `--best` options do not apply in paired-end mode.
        # Paired-end alignments where one mate's alignment is entirely contained within the other's are considered invalid.

        
        # first, align to ladder index to subtract any ladder oligos that were cloned        
    bowtie_ladder = bowtie_command %(
        path_l,
        fname+'_nomatch.fastq',
        path_l,
        fname+'_multi.fastq',
        path_l,
        fname+'_match.fastq',
        '/home/kazuki/^data_analysis/^scripts/ABNG_scripts/bowtie/indexes/ladder/ladder',
        inpath,
        fname + bowtie_input_extension,
        path_temp,
        fname + '_ladder_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n\n'+fname+'\n'+bowtie_ladder+'\n')
    logs.close()
    ###
    os.system(bowtie_ladder)        
        
        
        # second, align to tRNA index
    bowtie_tRNA = bowtie_command %(
        path_t,
        fname+'_nomatch.fastq',
        path_t,
        fname+'_multi.fastq',
        path_t,
        fname+'_match.fastq',
        '/home/kazuki/^data_analysis/^scripts/ABNG_scripts/bowtie/indexes/MG1655/Ec_tRNA',
        path_l,
        fname +'_nomatch.fastq',
        path_temp,
        fname + '_tRNA_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_tRNA+'\n')
    logs.close()
    ###
    os.system(bowtie_tRNA)        
        
        
        # third, align to the rRNA index
    bowtie_rRNA = bowtie_command %(
        path_r,
        fname+'_nomatch.fastq',
        path_r,
        fname+'_multi.fastq',
        path_r,
        fname+'_match.fastq',
        '/home/kazuki/^data_analysis/^scripts/ABNG_scripts/bowtie/indexes/MG1655/rRNA_MG1655',
        path_t,
        fname +'_nomatch.fastq',
        path_temp,
        fname + '_rRNA_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_rRNA+'\n')
    logs.close()
    ###
    os.system(bowtie_rRNA)         
        
        # finally align to the chr index
    bowtie_chr = bowtie_command %(
        path_chr,
        fname+'_nomatch.fastq',
        path_chr,
        fname+'_multi.fastq',
        path_chr,
        fname+'_match.fastq',
        '/home/kazuki/^data_analysis/^scripts/ABNG_scripts/bowtie/indexes/MG1655/e_coli_MG1655',
        path_r,
        fname +'_nomatch.fastq',
        path_chr,
        fname + '_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_chr+'\n')
    logs.close()
    ###
    os.system(bowtie_chr)
    

logs = open(log_file, 'a+')
present = str(datetime.now())
logs.write('\n'+present+'\n')
logs.close()
    
print 
print 
for line in open(log_file, 'r'):
    print line[:-1]

ks5
ks6
ks7
ks8
ks9
ks10
ks11
ks12
ks13
ks14
ks15
ks16
ks17
ks18
ks19
ks20
ks21
ks22


2019-06-03 09:37:13.393750


ks5
bowtie -v 2 -y -m 1 -a --best --strata -S -p 2 --un /home/kazuki/^profilingdata/20160420_reanalyze/alignments/ladder/ks5_nomatch.fastq --max /home/kazuki/^profilingdata/20160420_reanalyze/alignments/ladder/ks5_multi.fastq --al /home/kazuki/^profilingdata/20160420_reanalyze/alignments/ladder/ks5_match.fastq /home/kazuki/^data_analysis/^scripts/ABNG_scripts/bowtie/indexes/ladder/ladder /home/kazuki/^profilingdata/20160420_reanalyze/ks5-trimmed.fastq /home/kazuki/^profilingdata/20160420_reanalyze/tmpds/ks5_ladder_match.SAM 1>> /home/kazuki/^profilingdata/20160420_reanalyze/log_bowtie_ks48.txt 2>> /home/kazuki/^profilingdata/20160420_reanalyze/log_bowtie_ks48.txt
# reads processed: 14420851
# reads with at least one reported alignment: 58816 (0.41%)
# reads that failed to align: 14352580 (99.53%)
# reads with alignments suppressed due to -m: 9455 (0.07%)
Reported 58816 al

In [15]:
######################################################
import csv
import pickle
from copy import deepcopy

out_path=filepath+'density/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files:
    samgenmain_filename=filepath+'alignments/chr/'+f+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    gff = '/home/kazuki/^data_analysis/MG1655ver2/coli.gff'

    three_mapped=three_end_mapping(samgenmain,gff)
    pickle_readcounts = out_path+f+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)

    three_mapped_rpm = deepcopy(three_mapped)
    readcounts_to_rpm(three_mapped_rpm[0])
    readcounts_to_rpm(three_mapped_rpm[1])
    pickle_rpm = out_path+f+mapping+'_rpm.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)
        
    density_path = out_path+f+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_minus")

In [4]:
# make a python-dictionary of RPKMs from each library
import os
import pickle
from pandas import Series, DataFrame
import pandas as pd

in_path = filepath+'density/'
out_path = filepath+'expression_2/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

#########################################################################################

with open('/home/kazuki/^data_analysis/^Ecoli_Genome/Ecoli_Gene_TU.pickle','rb') as GT_f:
    Gene_TU_dict = pickle.load(GT_f)

for f in files:
    expression_dict={}
    out_pickle=out_path+f+mapping+gene_type+'_expression.pickle'
    out_csv=out_path+f+mapping+gene_type+'_expression.csv'
    readcounts_file = in_path+f+mapping+'_readcounts.pickle'
    
    with open(readcounts_file, 'rb') as rf:
        readcounts_dict = pickle.load(rf)
    plus_readcounts = readcounts_dict[0][readcounts_dict[0].keys()[0]]
    minus_readcounts = readcounts_dict[1][readcounts_dict[1].keys()[0]]
    total_read = sum(plus_readcounts)+sum(minus_readcounts)
    
    for key in Gene_TU_dict.keys():
        if key in expression_dict.keys():
            #print key +' is overlapping!!!'
            continue
    
        ########           
        if Gene_TU_dict[key][0] == '+':
            start = int(Gene_TU_dict[key][2])-1+15
            end = int(Gene_TU_dict[key][3])
        if Gene_TU_dict[key][0] == '-':
            start = int(Gene_TU_dict[key][2])-1
            end = int(Gene_TU_dict[key][3])-15            
        ########  
        
        length = end-start
        if length <= 0:
            #print key + ' is shorter than zero!!!'
            continue
        if Gene_TU_dict[key][0] == '+':
            reads = float(sum(plus_readcounts[start:end]))
        if Gene_TU_dict[key][0] == '-':
            reads = float(sum(minus_readcounts[start:end]))
        rpc = (reads/length)*3
        rpkm = (((reads/total_read)*1000000)/length)*1000
        expression_dict[key]=[]
        expression_dict[key].append(reads)
        expression_dict[key].append(rpc)
        expression_dict[key].append(rpkm)
    
    with open(out_pickle, 'wb') as op:
        pickle.dump(expression_dict,op)           
    
    rpkm_DataFrame = DataFrame.from_dict(expression_dict, orient='index')
    rpkm_DataFrame.columns = ['reads','rpc','rpkm']
    rpkm_DataFrame.to_csv(out_csv) 

In [5]:
print "good"

good


In [6]:
######################################
########### RPKMs for 3UTR ###########
######################################

import os
import pickle
from pandas import Series, DataFrame
import pandas as pd

in_path = filepath+'density/'
out_path = filepath+'expression_3UTR/' ##################
if not os.path.exists(out_path):
    os.makedirs(out_path)

#########################################################################################

with open('/home/kazuki/^data_analysis/^Ecoli_Genome/Ecoli_Gene_TU.pickle','rb') as GT_f:
    Gene_TU_dict = pickle.load(GT_f)

for f in files:
    expression_dict={}
    out_pickle=out_path+f+mapping+gene_type+'_expression.pickle'
    out_csv=out_path+f+mapping+gene_type+'_expression.csv'
    readcounts_file = in_path+f+mapping+'_readcounts.pickle'
    
    with open(readcounts_file, 'rb') as rf:
        readcounts_dict = pickle.load(rf)
    plus_readcounts = readcounts_dict[0][readcounts_dict[0].keys()[0]]
    minus_readcounts = readcounts_dict[1][readcounts_dict[1].keys()[0]]
    total_read = sum(plus_readcounts)+sum(minus_readcounts)
    
    for key in Gene_TU_dict.keys():
        if key in expression_dict.keys():
            #print key +' is overlapping!!!'
            continue
            
            
        ########
        if type(Gene_TU_dict[key][4]) != str:
            continue

        TheLASTGene = Gene_TU_dict[key][4].split('-')[-1]
        if key != TheLASTGene:
            continue
            
        if Gene_TU_dict[key][0] == '+':
            start = int(Gene_TU_dict[key][3])-1+15
            end = int(Gene_TU_dict[key][6])
        if Gene_TU_dict[key][0] == '-':
            start = int(Gene_TU_dict[key][5])-1
            end = int(Gene_TU_dict[key][2])-15
        ########
        
        
        length = end-start
        if length <= 0:
            #print key + ' is shorter than zero!!!'
            continue
        if Gene_TU_dict[key][0] == '+':
            reads = float(sum(plus_readcounts[start:end]))
        if Gene_TU_dict[key][0] == '-':
            reads = float(sum(minus_readcounts[start:end]))
        rpc = (reads/length)*3
        rpkm = (((reads/total_read)*1000000)/length)*1000
        expression_dict[key]=[]
        expression_dict[key].append(reads)
        expression_dict[key].append(rpc)
        expression_dict[key].append(rpkm)
    
    with open(out_pickle, 'wb') as op:
        pickle.dump(expression_dict,op)           
    
    rpkm_DataFrame = DataFrame.from_dict(expression_dict, orient='index')
    rpkm_DataFrame.columns = ['reads','rpc','rpkm']
    rpkm_DataFrame.to_csv(out_csv) 

In [7]:
print "good"

good
